<a href="https://colab.research.google.com/github/euredog/hackeps-2021/blob/neat/ProvaTensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

In [4]:
def get_input_data(water_route: str, air_file: str, amoni_file: str):
	# Load data
	data_water = pd.read_csv(water_route)
	data_air = pd.read_csv(air_file)
	data_amoni = pd.read_csv(amoni_file)
	# Join Data
	result = data_water.merge(data_air, on='row_date', how='outer', suffixes=('_air', '_water'))
	result = result.merge(data_amoni, on='row_date').dropna()
	# Split data
	inputs = result[['value_water', 'value_air', 'value']]
	outputs = result[['is_drift', 'dangerous_drift']].astype(int)
	return inputs.to_numpy(), outputs.to_numpy()

In [ ]:
inputs, outputs = get_input_data("/content/aigua.csv", "/content/aire.csv", "/content/amoni.csv")
x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=.2)
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

In [34]:
model = Sequential()
model.add(Dense(units=3, activation='tanh', input_shape=(3,)))
model.add(Dense(units=8, activation='tanh'))
model.add(Dense(units=4, activation='tanh'))
model.add(Dense(units=2, activation='tanh'))

In [35]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=32)

In [40]:
predictions = model.predict(x_test)
preds = []
for row in predictions:
    row = [0 if val<0.5 else 1 for val in row]
    preds.append(row)

In [ ]:
accuracy_score(y_test, preds)